In [20]:
import numpy as np

In [21]:
# path = "/home/xiao/Projects/predrnn-pytorch/data/moving-mnist-example/moving-mnist-test.npz"
# data = {}
# dat_1 = np.load(path)
# for key in dat_1.keys():
#     print ('key', key)
#     data[key] = dat_1[key]
# print ('data[clips]', data['clips'], data['clips'].shape)
# print ('data[dims]', data['dims'])
# print ('data[input_raw_data]', data['input_raw_data'], data['input_raw_data'].shape, np.median(data['input_raw_data']), np.max(data['input_raw_data']))

# path = "/home/xiao/Projects/ST_LSTM_MP/Data/2D_map/Map_array/normalized_train_video.npz"
# data = np.load(path)
# for key in data.keys():
#     print ('key', key)

# print ('data[clips]', data['clips'].shape)
# print ('data[dims]', data['dims'])
# print ('data[input_raw_data]', data['input_raw_data'].shape, np.median(data['input_raw_data']), np.max(data['input_raw_data']))


In [22]:
import os.path as op
arr_dir = "../../Data/2D_map/Map_array"
total_maz = np.load(op.join(arr_dir, 'maz.npy'))
print ("111")

111


In [4]:
train_file = op.join(arr_dir, "train_path.npy")
train_path = np.load(train_file)

print ('total_maz', total_maz.shape)
print ('train_path', train_path.shape)


total_maz (120, 128, 128)
train_path (160000, 22, 2)


In [5]:
train_path = np.reshape(train_path, (100, -1, train_path.shape[1], train_path.shape[2]))
print ('train_path', train_path.shape, np.max(train_path), np.min(train_path))

train_path (100, 1600, 22, 2) 127 0


In [6]:
train_video = []
for maz_id in range(train_path.shape[0]):
    video_per_maz = []
#     for task_id in range(train_path.shape[1]):
    for task_id in range(1000):
        start, goal = train_path[maz_id][task_id][0], train_path[maz_id][task_id][1]
        maz = np.copy(total_maz[maz_id])
        maz[start] = 64
        maz[goal] = 192
        repeated_maz = np.tile(maz, (train_path.shape[2], 1))
        repeated_maz = np.reshape(repeated_maz, (train_path.shape[2], maz.shape[0], maz.shape[1]))
#         print (repeated_maz.shape)
        for path_id in range(1, train_path.shape[2]-1):
            pos = train_path[maz_id][task_id][path_id]
#             print ('pos', pos)
#             print ('repeated_maz[path_id:-1]', repeated_maz[path_id:-1].shape)
            repeated_maz[path_id:, pos] = 128
#             for i in range(path_id, )
        video_per_maz.append(repeated_maz)
    video_per_maz = np.array(video_per_maz)
#     print ('video_per_maz', video_per_maz.shape)
    train_video.append(video_per_maz)
train_video = np.stack(train_video)
print ('train_video', train_video.shape)
video_num = train_video.shape[0] * 1000
video_len = train_video.shape[2]

np.save(op.join(arr_dir, "train_video.npy"), train_video)



train_video (100, 1000, 22, 128, 128)


In [7]:
train_video = np.reshape(train_video, (-1, 1, train_video.shape[3], train_video.shape[4])).astype(np.float32)
print ('train_video', train_video.shape)
train_video = train_video/255.0
dims = np.array([1, train_video.shape[-2], train_video.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 10
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_train_video.npz"), dims=dims, clips=clips, input_raw_data=train_video)



train_video (2200000, 1, 128, 128)
(2, 100000, 2)


In [23]:
test_seen_file = op.join(arr_dir, "test_seen_path.npy")
test_seen_path = np.load(test_seen_file)
print ('test_seen_path', test_seen_path.shape)

test_seen_path = np.reshape(test_seen_path, (100, -1, test_seen_path.shape[1], test_seen_path.shape[2]))
print ('test_seen_path', test_seen_path.shape)

test_seen_path (40000, 19, 2)
test_seen_path (100, 400, 19, 2)


In [24]:
test_seen_video = []
for maz_id in range(test_seen_path.shape[0]):
    video_per_maz = []
#     for task_id in range(test_seen_path.shape[1]):
    for task_id in range(200):
        start, goal = test_seen_path[maz_id][task_id][0], test_seen_path[maz_id][task_id][1]
        maz = np.copy(total_maz[maz_id])
        maz[start[0], start[1]] = 64 #XXXXX
        maz[goal[0], goal[1]] = 192 #XXXXX
        repeated_maz = np.tile(maz, (test_seen_path.shape[2], 1))
#         print ('tile', repeated_maz.shape)
        repeated_maz = np.reshape(repeated_maz, (test_seen_path.shape[2], maz.shape[0], maz.shape[1]))
#         print ('reshape', repeated_maz.shape)
#         
        for path_id in range(1, test_seen_path.shape[2]-1):
            pos = test_seen_path[maz_id][task_id][path_id]
#             print ('pos', pos)
#             print ('repeated_maz[path_id:-1]', repeated_maz[path_id:-1].shape)
            repeated_maz[path_id:-1, pos[0], pos[1]] = 128 #XXXXX
        video_per_maz.append(repeated_maz)
    video_per_maz = np.array(video_per_maz)
    test_seen_video.append(video_per_maz)
test_seen_video = np.array(test_seen_video)
print ('test_seen_video', test_seen_video.shape)
# video_num = test_seen_video.shape[0] * test_seen_video.shape[1]
video_num = test_seen_video.shape[0] * 200
video_len = test_seen_video.shape[2]
np.save(op.join(arr_dir, "test_seen_video.npy"), test_seen_video)
    

test_seen_video (100, 200, 19, 128, 128)


In [19]:
test_seen_video = np.reshape(test_seen_video, (-1, 1, test_seen_video.shape[3], test_seen_video.shape[4])).astype(np.float32)
test_seen_video = test_seen_video/255.0
dims = np.array([1, test_seen_video.shape[-2], test_seen_video.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_seen_video.npz"), dims=dims, clips=clips, input_raw_data=test_seen_video)


(2, 20000, 2)


KeyboardInterrupt: 

In [11]:
test_unseen_file = op.join(arr_dir, "test_unseen_path.npy")
test_unseen_path = np.load(test_unseen_file)
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path = np.reshape(test_unseen_path, (20, -1, test_unseen_path.shape[1], test_unseen_path.shape[2]))
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path (8000, 17, 2)
test_unseen_path (20, 400, 17, 2)


In [12]:
test_unseen_video = []
for maz_id in range(test_unseen_path.shape[0]):
    video_per_maz = []
    for task_id in range(test_unseen_path.shape[1]):
        start, goal = test_unseen_path[maz_id][task_id][0], test_unseen_path[maz_id][task_id][1]
        maz = np.copy(total_maz[100 + maz_id])
        maz[start[0],start[1]] = 64
        maz[goal[0],goal[1]] = 192 #XXXXXX
        repeated_maz = np.tile(maz, (test_unseen_path.shape[2], 1))
        repeated_maz = np.reshape(repeated_maz, (test_unseen_path.shape[2], maz.shape[0], maz.shape[1]))
#         print (repeated_maz.shape)
        for path_id in range(1, test_unseen_path.shape[2]-1):
            pos = test_unseen_path[maz_id][task_id][path_id]
#             print ('pos', pos)
#             print ('repeated_maz[path_id:-1]', repeated_maz[path_id:-1].shape)
            repeated_maz[path_id:-1, pos] = 128
#             for i in range(path_id, )
        video_per_maz.append(repeated_maz)
    video_per_maz = np.array(video_per_maz)
    test_unseen_video.append(video_per_maz)
test_unseen_video = np.array(test_unseen_video)
print ('test_unseen_video', test_unseen_video.shape)
video_num = test_unseen_video.shape[0] * test_unseen_video.shape[1]
video_len = test_unseen_video.shape[2]
np.save(op.join(arr_dir, "test_unseen_video.npy"), test_unseen_video)
    

train_video (20, 400, 17, 128, 128)


In [13]:
test_unseen_video = np.reshape(test_unseen_video, (-1, 1, test_unseen_video.shape[3], test_unseen_video.shape[4])).astype(np.float32)
test_unseen_video = test_seen_video/255.0
dims = np.array([1, test_unseen_video.shape[-2], test_unseen_video.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_unseen_video.npz"), dims=dims, clips=clips, input_raw_data=test_unseen_video)


(2, 8000, 2)
